In [10]:
#step1: calculate video duration 

import cv2
import numpy as np
#from visualize_cv2 import model, display_instances, class_names




In [11]:

v = cv2.VideoCapture('test.mp4')
v.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
duration = v.get(cv2.CAP_PROP_POS_MSEC)

timestamp_each_frame = duration/60


print('duration (S) = ' + str(duration))
print('duration (S) = ' + str(timestamp_each_frame))

frameCount = int(v.get(cv2.CAP_PROP_FRAME_COUNT))
print('number of frames = ' + str(frameCount))

# the 1st frame is frame 0, not 1, so "5335" means after the last frame
POS_FRAMES = v.get(cv2.CAP_PROP_POS_FRAMES)
FRAME_COUNT = v.get(cv2.CAP_PROP_FRAME_COUNT)
print('POS_FRAMES = ' + str(POS_FRAMES))
print('FRAME_COUNT = ' + str(FRAME_COUNT))

v.release()

duration (S) = 8441.766666666666
duration (S) = 140.6961111111111
number of frames = 253
POS_FRAMES = 253.0
FRAME_COUNT = 253.0


In [5]:
#step2: output video frame time ,frame content and bounding box position in a dictionary

In [7]:
import cv2
import numpy as np
from visualize_cv2 import model, display_instances, class_names
import json




In [26]:
capture = cv2.VideoCapture('test.mp4')
size = (
    int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
codec = cv2.VideoWriter_fourcc(*'DIVX')
output = cv2.VideoWriter('videofile_masked2.avi', codec, 1, size)

count = 0
out_dict = {}

while(capture.isOpened()):
    # cap.read() returns a bool (True/False). 
    # If frame is read correctly, it will be True. So you can check end of the video by checking this return value
    ret, frame = capture.read()
    if ret:
        # add mask to frame
        results = model.detect([frame], verbose=0)
        r = results[0]
        frame = display_instances(
            frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
        )
        output.write(frame)
        cv2.imshow('frame', frame)

        # put in dict
        # frame_excel = video length / 1/60 : r['class_ids']
        # CSV
        #https://www.programiz.com/python-programming/nested-dictionary
        #https://stackoverflow.com/questions/10756427/loop-through-all-nested-dictionary-values
        #output object: out_dict[count] = r['class_ids']
        #wrong: out_dict[count][r['class_ids']] = r['rois']
        #output bunding box: out_dict[count] = {}
        #for i, cls_id in enumerate(r['class_ids']):  
        #    out_dict[count][cls_id] = r['rois'][i, :]
        
        out_dict[count] = {}
        out_dict[count]['cls_id'] = r['class_ids']
        out_dict[count]['rois'] = r['rois']
        
        
        
 
        count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

capture.release()
output.release()
cv2.destroyAllWindows()


In [ ]:
#Step3: put all the video frame time ,frame content and bounding box position in a JSON file

In [42]:
each_time_interval = duration / (frameCount-1)

In [43]:
import collections
frame2content = {}
for k, v in out_dict.items():
    objs = [class_names[i] for i in v["cls_id"]]
    obj2num = dict(collections.Counter(objs))
    _objs = []
    for o in objs:
        _objs.append(o + "_" + str(obj2num[o]))
        obj2num[o] -= 1
        
    obj2roi = {}
    for idx, o in enumerate(_objs):
        obj2roi[o] = list(map(int, list(v["rois"][idx, :])))
        #obj2roi[o] = v["rois"][idx, :]
    frame2content[k * each_time_interval] = obj2roi
        
    

In [44]:
print(frame2content)

{0.0: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [107, 0, 238, 472]}, 33.49907407407407: {'car_1': [3, 114, 75, 205], 'chair_1': [15, 61, 46, 86], 'bench_1': [98, 0, 235, 480]}, 66.99814814814815: {'car_1': [3, 114, 75, 205], 'chair_1': [14, 61, 46, 87]}, 100.49722222222222: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 88]}, 133.9962962962963: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [104, 0, 237, 448]}, 167.49537037037038: {'car_1': [5, 113, 74, 204], 'chair_1': [15, 62, 46, 86], 'truck_1': [17, 234, 73, 306], 'bench_1': [106, 2, 234, 444]}, 200.99444444444444: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [106, 2, 234, 443]}, 234.4935185185185: {'car_1': [5, 113, 74, 204], 'chair_1': [15, 61, 47, 87], 'bench_1': [111, 0, 230, 453], 'truck_1': [17, 234, 73, 306]}, 267.9925925925926: {'car_1': [3, 114, 75, 205], 'chair_1': [15, 62, 45, 86], 'bench_1': [104, 0, 234, 475]}, 301.4916666666667: {'car_2': [

In [45]:
import json
with open('nik2.json', 'w') as fp:
   json.dump(frame2content, fp, indent=2)